In [12]:
import random
import math

def calculate_distance(city1, city2):
    """
    Calculates the Euclidean distance between two cities.

    Parameters:
    city1 (tuple): A tuple of two integers representing the coordinates of city1.
    city2 (tuple): A tuple of two integers representing the coordinates of city2.

    Returns:
    float: The Euclidean distance between city1 and city2.
    """

    # Calculate the difference between the x-coordinates of the two cities, square it,
    # add it to the square of the difference between the y-coordinates, and take the square root.
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)


def calculate_distance_matrix(cities):
    """
        Generates a distance matrix from a list of cities and their coordinates.

        Parameters:
        cities (list): A list of city names and their coordinates.

        Returns:
        list: A distance matrix of pairwise distances between cities.
        """
    n = len(cities)  # Get the number of cities.

    # Create an empty distance matrix with dimensions n x n.
    distance_matrix = [[0] * n for _ in range(n)]

    # Loop over each pair of cities and calculate their distance.
    for i in range(n):
        # Only calculate the distance once for each pair of cities.
        for j in range(i+1, n):
            # Calculate the distance between cities i and j using the calculate_distance function.
            distance = calculate_distance(cities[i], cities[j])
            # Set the (i, j) entry to the distance.
            distance_matrix[i][j] = distance
            # Set the (j, i) entry to the distance (since the distance between cities i and j is the same as the distance between cities j and i).
            distance_matrix[j][i] = distance

    return distance_matrix  # Return the distance matrix.


def randomized_nearest_neighbor(cities, distance_matrix):
    """
    Generates an initial route using the randomized nearest neighbor algorithm.

    Parameters:
    cities (list): A list of city names.
    distance_matrix (list): A distance matrix of pairwise distances between cities.

    Returns:
    list: A route generated by the randomized nearest neighbor algorithm.
    """
    n = len(cities)   # Get the number of cities
    # Initialize visited_cities list with n number of False
    visited_cities = [False] * n
    route = [0] * n   # Initialize route list with n number of 0's

    # Select a random starting city.
    # Select a random city index as the starting city
    current_city = random.randint(0, n-1)
    visited_cities[current_city] = True   # Mark the starting city as visited
    # Add the starting city to the first index of route
    route[0] = current_city

    # Add the remaining cities to the route.
    for i in range(1, n):   # Iterate through the remaining cities
        # Compute the distances from the last city added to all remaining cities.
        distances = [(distance_matrix[current_city][j], j)
                     for j in range(n) if not visited_cities[j]]   # Calculate the distance between the current city and all unvisited cities
        # Select the city with the shortest distance to the last city and add it to the route.
        # Get the city with the shortest distance from the current city
        min_distance, nearest_city = min(distances)
        # Mark the selected city as visited
        visited_cities[nearest_city] = True
        route[i] = nearest_city   # Add the selected city to the route list
        current_city = nearest_city   # Update the current city to the selected city

    return route   # Return the generated route list


def calculate_total_distance(route, distance_matrix):
    """
    Calculates the total distance of a route given its distance matrix.

    Parameters:
    route (list): A list of city indices representing the route.
    distance_matrix (list): A distance matrix of pairwise distances between cities.

    Returns:
    float: The total distance of the given route.
    """
    total_distance = 0  # initialize total distance to zero
    n = len(route)  # get the number of cities in the route
    for i in range(n-1):  # iterate over the cities in the route (excluding the last one)
        # add the distance between the current city and the next city in the route to the total distance
        total_distance += distance_matrix[route[i]][route[i+1]]
    # add the distance between the last city in the route and the first city in the route to the total distance
    total_distance += distance_matrix[route[-1]][route[0]]
    return total_distance  # return the total distance


def swap_cities(route, i, j):
    """
    Swaps two cities in a route.

    Parameters:
    route (list): A list of city indices representing the route.
    i (int): The index of the first city to be swapped.
    j (int): The index of the second city to be swapped.

    Returns:
    list: A new list with the two cities swapped.
    """
    # Make a copy of the original route list.
    new_route = route[:]
    # Swap the cities at indices i and j.
    new_route[i], new_route[j] = new_route[j], new_route[i]
    return new_route


def apply_2opt_swap(route, i, j):
    """
    Applies the 2-opt swap to a route.

    Given a route and two indices i and j, this function applies a 2-opt swap to
    the portion of the route between indices i and j. Specifically, it reverses
    the order of the nodes between i and j (inclusive) and returns the new route.
    This can be used in algorithms that optimize routes, such as the Traveling
    Salesman Problem.

    Parameters:
    route (list): A list of nodes representing a route.
    i (int): The starting index of the portion of the route to swap.
    j (int): The ending index of the portion of the route to swap.

    Returns:
    list: A new list representing the route with the 2-opt swap applied.
    """
    # create a new route by concatenating three slices of the original route:
        # - the portion of the route before index i
        # - the portion of the route between indices i and j, reversed
        # - the portion of the route after index j
    new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
    # return the new route
    return new_route


def run_tsp(cities, num_iterations):
    """
    Runs the Traveling Salesman Problem algorithm using RNNA and2-opt optimization.

    Args:
    cities (list): A list of cities.
    num_iterations (int): The number of iterations to perform.

    Returns:
    None. Prints the initial route and distance, the RNNA route and distance, and the optimized route and distance.
    """

    # Calculate the distance matrix between cities.
    distance_matrix = calculate_distance_matrix(cities)

    # Generate an initial route using random shuffling.
    initial_route = list(range(len(cities)))
    random.shuffle(initial_route)
    initial_distance = calculate_total_distance(initial_route, distance_matrix)

    # Generate a route using the randomized nearest neighbor algorithm.
    rnna_route = randomized_nearest_neighbor(cities, distance_matrix)
    rnna_distance = calculate_total_distance(rnna_route, distance_matrix)

    # Apply the 2-opt algorithm to improve the route.
    current_best_route = rnna_route
    current_best_distance = rnna_distance

    # Perform a given number of iterations and apply the 2-opt swap to improve the route.
    for i in range(num_iterations):
        # Select two random cities in the current best route.
        city1, city2 = random.sample(range(len(initial_route)), 2)
        # Apply the 2-opt swap to the current best route.
        new_route = apply_2opt_swap(current_best_route, city1, city2)

        # Calculate the total distance of the new route.
        new_distance = calculate_total_distance(new_route, distance_matrix)

        # If the new route is shorter than the current best route, update the current best route.
        if new_distance < current_best_distance:
            current_best_route = new_route
            current_best_distance = new_distance

    # Print the initial route and distance.
    print("Route list output is limited to 15 elements only for readability.")

    print("\nInitial Route: ", initial_route[0:14])
    print("Initial Distance: ", initial_distance, "km")

    # Print the RNNA route and distance, and the improvement by RNNA.
    print("\nRandomized Nearest Neighbor Route: ", rnna_route[0:14])
    print("Randomized Nearest Neighbor Distance: ", rnna_distance, "km")
    print("Improved by RNNA: -", initial_distance - rnna_distance, "km | In percentage: ",
          (initial_distance - rnna_distance) / initial_distance * 100, "%")

    # Print the optimized route and distance, and the improvement by 2-opt swaps.
    print("\nOptimized Route: ", current_best_route[0:14])
    print("Optimized Distance: ", current_best_distance, "km")
    print("Improved by 2-Opt Swap: -", rnna_distance - current_best_distance,
          "km | In percentage: ", (rnna_distance - current_best_distance) / rnna_distance * 100, "%")


In [11]:
# TEST CASE 1 - 15 cities; 1000 iterations
# Original proposed solution to the vaccine distribution problem
# Carl's note: I wanted to use the real location of the cities in Quezon, but due to time constraints and conflicting priorities, I used random coordinates. I wish to implement this in the future.

num_iterations = 1000
cities = [(110.0, 225.0), (161.0, 283.0), (188.0, 293.0), (264.0, 292.0),    (280.0, 231.0), (333.0, 198.0), (340.0, 149.0),
          (366.0, 129.0),    (382.0, 179.0), (380.0, 218.0), (394.0, 261.0), (413.0, 242.0),    (449.0, 269.0), (477.0, 246.0), (469.0, 205.0)]

run_tsp(cities, num_iterations)

Initial Route:  [0, 10, 14, 4, 3, 12, 9, 1, 13, 11]
Initial Distance:  2410.8185328092045 km

Randomized Nearest Neighbor Route:  [10, 11, 9, 8, 6, 7, 5, 4, 3, 2]
Randomized Nearest Neighbor Distance:  1088.6094614124058 km
Improved by RNNA: - 1322.2090713967987 km | In percentage:  54.844819442137585 %

Optimized Route:  [14, 13, 12, 11, 10, 3, 2, 1, 0, 4]
Optimized Distance:  961.8890229924045 km
Improved by 2-Opt Swap: - 126.7204384200013 km | In percentage:  11.640578454608422 %


In [10]:
# TEST CASE 2 - 1000 cities; 10000 iterations
# Scaled up version of the original proposed solution to the vaccine distribution problem.
# Wider spread of cities.
# More iterations to improve the route.

num_iterations = 10000
# Generate 1000 cities with random coordinates from 0 - 1000 km.
cities = [(random.randint(0, 1000), random.randint(0, 1000))
          for i in range(1000)]

run_tsp(cities, num_iterations)


Initial Route:  [463, 844, 765, 554, 819, 826, 204, 259, 605, 255]
Initial Distance:  531685.0307905608 km

Randomized Nearest Neighbor Route:  [27, 410, 751, 517, 125, 523, 518, 622, 107, 445]
Randomized Nearest Neighbor Distance:  29520.400486374714 km
Improved by RNNA: - 502164.630304186 km | In percentage:  94.44776535414567 %

Optimized Route:  [531, 838, 831, 115, 203, 865, 839, 619, 796, 268]
Optimized Distance:  29364.500510001933 km
Improved by 2-Opt Swap: - 155.89997637278066 km | In percentage:  0.5281092864737287 %
